<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import librosa
import os
import csv
from scipy import signal 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# '/home/stattus4dpenalva/stattus4/spectrum_filter_fft'
def readfilter(path_filter):
  '''
    read file with spectral filter and output vector with filter
  '''
  line =[]
  with open(path_filter,'r') as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:
      line.append(row)
      if len(line[-1]) == 1:
        print(line[-1])
      else:
        filter_spec = np.array(np.float32(line[-1]) )
  return filter_spec



def load_binary_tagged_data(data_path,tag):
  '''
   Data
  '''  
  nomesref = [f for f in os.listdir(data_path)]
  data_fs=[]
  nome = []
  for f in nomesref:
    if f[0:2] == tag:
      data,fs = librosa.load(data_path+f,sr=None)
      data_fs.append(data)
      nome.append(f)
    
  return data_fs,nome

def filtro_spect(data_path,tag, filtered_output_path = '/home/stattus4dpenalva/stattus4/filterspec_sv_data/',filter_norm = 'size', save_filter = True, train = True, spect_filter = '/home/stattus4dpenalva/stattus4/spectrum_filter_fft'):
  '''
     Filtro de ruido espectral do sensor de signal, na hipótese de todas medidas sem vazamente apresentarem mesmas
   componentes espectrais provenientes do sensor digital de signal. Amostras são consideradas de mesmo tamanho.
   
   filter_norm: 'size' for the size of the frequency domain, other case will default for the sum of the coeficients
  '''
  data_fs,nome = load_binary_tagged_data(data_path,tag)
  fft_normalized_signal = []
  fftnorms = []
  ldata = len(data_fs[0])
  if train:
    freq_hist = np.zeros(shape=ldata)
  else:
    freq_hist = readfilter(spect_filter)
    freq_hist = freq_hist[:ldata]
    
  signalfreq = []
  for i in range(len(data_fs)):
    signalfreq.append(np.angle( np.fft.fft(data_fs[i],n=ldata) ))
    svfreq = np.abs(np.fft.fft(data_fs[i],n=ldata))  
    fftnorms.append(np.sum(svfreq)) 
    svfreq /= fftnorms[-1]  
    fft_normalized_signal.append(svfreq)
    

  if train:
    for i in range(len(data_fs)):
      freq_hist = freq_hist + fft_normalized_signal[i]*fftnorms[i]

  if filter_norm == 'size':
    freq_hist_N = freq_hist/len(freq_hist)
    freq_hist = freq_hist_N
  else:
    freq_hist /= np.sum(freq_hist)    
    
  if save_filter:
    if filter_norm == 'size':      
      with open('/home/stattus4dpenalva/stattus4/spectrum_filter_fft','w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(['filter_size_normed'])
        writer.writerow(freq_hist_N)  
    else:
      with open('/home/stattus4dpenalva/stattus4/spectrum_filter_fft','w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(['filter_coeficient_normed'])
        writer.writerow(freq_hist)
      

  dist = []
  spec_filtered_list = []
  spec_filtered = np.zeros(shape=ldata)
  for i in range(len(data_fs)):
    d = fft_normalized_signal[i]-freq_hist
    d[d < 0.0] = 0.0  
    spec_filtered = d*fftnorms[i]*np.exp(signalfreq[i]*1j)      
    librosa.output.write_wav(filtered_output_path+'filtered_{}'.format(nome[i]),np.real(np.fft.ifft(spec_filtered)),fs)
   

In [ ]:
filtro_spect('/home/stattus4dpenalva/stattus4/wav_data_2018/','CV', filtered_output_path = '/home/stattus4dpenalva/stattus4/filterspec_cv_data/',filter_norm = 'size', save_filter = False, train = False, spect_filter = '/home/stattus4dpenalva/stattus4/spectrum_filter_fft')